# StochasticモデルによるMNISTのCNN学習

ネットワーク全体に Stochastic性が成り立つ前提で Stochasticモデルに基づくLUT回路学習を行います。<br> 
Stochastic計算については[こちら](https://en.wikipedia.org/wiki/Stochastic_computing)などを参照ください。

## 事前準備

In [1]:
import numpy as np
from tqdm.notebook import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

データセットは PyTorch の torchvision を使います。

今回はバイナリ化は行わずに、多値を尤度値として扱います。

In [2]:
# setting
epoch     = 4
net_name  = 'MnistStochasticLutCnn'
data_path = './data' + net_name

# dataset
dataset_train = torchvision.datasets.MNIST(root='./data/', train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root='./data/', train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=32, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=32, shuffle=False, num_workers=2)

## ネットワークの構築

DifferentiableLut の BatchNormalization や Binarize を無効化することで Stochastic 演算モデルとなります。

MaxPooling もデジタルにおける OR 演算を Stochastic計算に置き換えたものを使います。

In [3]:
# define network

lut_layer0_0 = bb.DifferentiableLut([192], batch_norm=False, binarize=False)
lut_layer0_1 = bb.DifferentiableLut([32], batch_norm=False, binarize=False)

lut_layer1_0 = bb.DifferentiableLut([192], batch_norm=False, binarize=False)
lut_layer1_1 = bb.DifferentiableLut([32], batch_norm=False, binarize=False)

lut_layer2_0 = bb.DifferentiableLut([256], batch_norm=False, binarize=False)
lut_layer2_1 = bb.DifferentiableLut([64], batch_norm=False, binarize=False)

lut_layer3_0 = bb.DifferentiableLut([256], batch_norm=False, binarize=False)
lut_layer3_1 = bb.DifferentiableLut([64], batch_norm=False, binarize=False)

lut_layer4_0 = bb.DifferentiableLut([1024], batch_norm=False, binarize=False)
lut_layer4_1 = bb.DifferentiableLut([360], batch_norm=False, binarize=False)
lut_layer4_2 = bb.DifferentiableLut([60], batch_norm=False, binarize=False)
lut_layer4_3 = bb.DifferentiableLut([10], batch_norm=False, binarize=False)


net = bb.Sequential([
            bb.Sequential([
                bb.Convolution2d(bb.Sequential([lut_layer0_0, lut_layer0_1]), filter_size=(3, 3)),
                bb.Convolution2d(bb.Sequential([lut_layer1_0, lut_layer1_1]), filter_size=(3, 3)),
                bb.StochasticMaxPooling(filter_size=(2, 2)),
            ]),
            bb.Sequential([
                bb.Convolution2d(bb.Sequential([lut_layer2_0, lut_layer2_1]), filter_size=(3, 3)),
                bb.Convolution2d(bb.Sequential([lut_layer3_0, lut_layer3_1]), filter_size=(3, 3)),
                bb.StochasticMaxPooling(filter_size=(2, 2)),
            ]),
            bb.Sequential([
                bb.Convolution2d(bb.Sequential([lut_layer4_0, lut_layer4_1, lut_layer4_2, lut_layer4_3]),
                                    filter_size=(4, 4)),
            ]),
        ])

net.set_input_shape([1, 28, 28])

net.send_command("binary false")       # バイナリ化しない(念のため)
net.send_command("lut_binarize true")  # LUTテーブル自体はバイナリ化する

## 学習の実施

load_networks/save_networks で途中結果を保存/復帰可能できます。ネットワークの構造が変わると正常に読み込めなくなるので注意ください。
(その場合は新しいネットをsave_networksするまで一度load_networks をコメントアウトください)

tqdm などを使うと学習過程のプログレス表示ができて便利です。

In [4]:
#bb.load_networks(data_path, net)

# learning
loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam()

optimizer.set_variables(net.get_parameters(), net.get_gradients())

for epoch in range(epoch):
    loss.clear()
    metrics.clear()

    # learning
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
            
            y_buf = net.forward(x_buf, train=True)
            
            dy_buf = loss.calculate(y_buf, t_buf)

            metrics.calculate(y_buf, t_buf)
            net.backward(dy_buf)
            
            optimizer.update()

            t.set_postfix(loss=loss.get(), acc=metrics.get())
    
    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    bb.save_networks(data_path, net)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))


epoch[0] : loss=1.688245 accuracy=0.757900



epoch[1] : loss=1.657572 accuracy=0.815700



epoch[2] : loss=1.638352 accuracy=0.830200



epoch[3] : loss=1.643433 accuracy=0.807500


## FPGA用RTL(Verilog)出力

FPGA合成の為のVerilogを出力します。

現状変換可能なのが、stride=1 の畳み込み層の連続＋最後に一個だけ MaxPooling という単位なので3つに分けて変換しています。<br>
返還後の Verilog はそれぞれ Xilinx の AXI4-Stream Video 規格に準じています(frame start で tuser がアサートされるビデオ信号)。

In [5]:
# export verilog
with open(net_name+'.v', 'w') as f:
    f.write('`timescale 1ns / 1ps\n\n')
    f.write(bb.make_verilog_lut_cnv_layers(net_name + 'Cnv0', net[0]))
    f.write(bb.make_verilog_lut_cnv_layers(net_name + 'Cnv1', net[1]))
    f.write(bb.make_verilog_lut_cnv_layers(net_name + 'Cnv2', net[2]))

## モデルの検証

今回のモデルは Stochastic 性がある前提で学習しています。そこで本当にそのままLUTにマップして認識能力があるか確認します。

BinaryBrain には BinaryLut という単なるバイナリテーブルを引くだけのモデルがあります。<br>
ここに学習結果をマッピングしてネットワークを作り認識可能か確認します。

なお、この際に前後に RealToBinary と BinaryToReal を挟んでバイナリ変調を施しています。<br>
閾値を変えることで多値の入力を確率的な0と1の配列に変えることで一般的な画像に対して Stochastic性を与えています<br>

これは実用的にはハイフレームレート(オーバーサンプリング)でノイズのある画像入力を直接または疑似的に用意すれば機能することを示しています。

In [6]:
# 学習したモデルを読み込み(念のため)
bb.load_networks(data_path, net)

# LUTモデルは BIT型を使ってメモリ節約が可能
bin_dtype = bb.DType.BIT  # bb.DType.BIT or bb.DType.FP32

# 同一形状のバイナリLUTを生成
bin_lut0_0 = bb.BinaryLut.from_sparse_model(lut_layer0_0, fw_dtype=bin_dtype)
bin_lut0_1 = bb.BinaryLut.from_sparse_model(lut_layer0_1, fw_dtype=bin_dtype)
bin_lut1_0 = bb.BinaryLut.from_sparse_model(lut_layer1_0, fw_dtype=bin_dtype)
bin_lut1_1 = bb.BinaryLut.from_sparse_model(lut_layer1_1, fw_dtype=bin_dtype)
bin_lut2_0 = bb.BinaryLut.from_sparse_model(lut_layer2_0, fw_dtype=bin_dtype)
bin_lut2_1 = bb.BinaryLut.from_sparse_model(lut_layer2_1, fw_dtype=bin_dtype)
bin_lut3_0 = bb.BinaryLut.from_sparse_model(lut_layer3_0, fw_dtype=bin_dtype)
bin_lut3_1 = bb.BinaryLut.from_sparse_model(lut_layer3_1, fw_dtype=bin_dtype)
bin_lut4_0 = bb.BinaryLut.from_sparse_model(lut_layer4_0, fw_dtype=bin_dtype)
bin_lut4_1 = bb.BinaryLut.from_sparse_model(lut_layer4_1, fw_dtype=bin_dtype)
bin_lut4_2 = bb.BinaryLut.from_sparse_model(lut_layer4_2, fw_dtype=bin_dtype)
bin_lut4_3 = bb.BinaryLut.from_sparse_model(lut_layer4_3, fw_dtype=bin_dtype)

# テスト用ネットワーク構築
frame_modulation_size = 7

test_net = bb.Sequential([
                bb.RealToBinary(frame_modulation_size=frame_modulation_size, bin_dtype=bin_dtype),
                bb.Convolution2d(bb.Sequential([bin_lut0_0, bin_lut0_1]), filter_size=(3, 3), fw_dtype=bin_dtype),
                bb.Convolution2d(bb.Sequential([bin_lut1_0, bin_lut1_1]), filter_size=(3, 3), fw_dtype=bin_dtype),
                bb.MaxPooling(filter_size=(2, 2), fw_dtype=bin_dtype),
                bb.Convolution2d(bb.Sequential([bin_lut2_0, bin_lut2_1]), filter_size=(3, 3), fw_dtype=bin_dtype),
                bb.Convolution2d(bb.Sequential([bin_lut3_0, bin_lut3_1]), filter_size=(3, 3), fw_dtype=bin_dtype),
                bb.MaxPooling(filter_size=(2, 2), fw_dtype=bin_dtype),
                bb.Convolution2d(bb.Sequential([bin_lut4_0, bin_lut4_1, bin_lut4_2, bin_lut4_3]), filter_size=(4, 4), fw_dtype=bin_dtype),
                bb.BinaryToReal(frame_modulation_size=frame_modulation_size, bin_dtype=bin_dtype)
            ])
test_net.set_input_shape([1, 28, 28])

#print(test_net.get_info())

# 推論評価
test_loss    = bb.LossSoftmaxCrossEntropy()
test_metrics = bb.MetricsCategoricalAccuracy()

loss.clear()
metrics.clear()
for images, labels in tqdm(loader_test):
    x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
    t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
    y_buf = test_net.forward(x_buf, train=False)
    test_loss.calculate(y_buf, t_buf)
    test_metrics.calculate(y_buf, t_buf)

print('Binary LUT test : loss=%f accuracy=%f' % (test_loss.get(), test_metrics.get()))

load : ./dataMnistStochasticLutCnn\20201229_125107



Binary LUT test : loss=1.671842 accuracy=0.766100
